<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/process_tWTWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import sys
import glob

import re
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
EMOJI_PATTERN = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [ ]:
text_processor = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone',
        'time', 'url', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},

    fix_html=True,
    segmenter="twitter",
    corrector="twitter",

    unpack_hashtags=True,
    unpack_contractions=True,
    spell_correct_elong=True,

    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    dicts=[emoticons]
)


/Users/siqiwu/opt/anaconda3/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/Users/siqiwu/opt/anaconda3/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
REMOVE_TAGS = [
    "<emphasis>", "<kiss>", "<repeated>", "<laugh>", "<allcaps>",
    "</allcaps>", "<angel>", "<elongated>", "<tong>", "<annoyed>",
    "<censored>", "<happy>", "<percent>", "<wink>",
    "<headdesk>", "<surprise>", "<date>", "<time>", "<url>",
    "<sad>", "<email>", "<phone>", "<hashtag>", "</hashtag>"
    ]

ADD_TO_GLOVE = ["<number>", "<money>"]


In [ ]:
REMOVE_PUNCTUATIONS = ["!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "-", "+", "_", "=", "{", "}", "[", "]", ":", ";", "'", '"', ",", ".", "?", "/"]

In [ ]:
# Try removing punctuations as well

def pre_process_single(tweet, t_id):
    tweet_toked_text = []
    de_emojified_text = tweet.encode('ascii', 'ignore').decode('ascii')
    de_emojified_text = re.sub(r"\$", " $ ", de_emojified_text)
    company_normalize_text = EMOJI_PATTERN.sub(r' ', de_emojified_text)

    tokens = text_processor.pre_process_doc(company_normalize_text)
    for token in tokens:
        if token in REMOVE_TAGS:
            continue
        if token in REMOVE_PUNCTUATIONS: #remove punctuations
            continue
        tweet_toked_text.append(str(token))

    return tweet_toked_text

In [ ]:
id2text = {}
tweet_acquired = 0
tweet_valid = 0
tweet_invalid = 0

In [ ]:
for fil in glob.glob("/Users/siqiwu/Documents/Berkeley/Info-NLP/project/bias-stance/data/wtwt/scrapped_full/*"):
    fo = open(fil, "r")
    full_tweet = json.load(fo)
    fo.close()
    tweet_acquired = tweet_acquired + 1
    if "data" not in full_tweet:
        tweet_invalid = tweet_invalid + 1
        continue
    tweet_valid = tweet_valid + 1
    tweet_id = full_tweet["data"][0]["id"]
    txt = pre_process_single(full_tweet["data"][0]["text"], tweet_id)
    if len(txt) > 0 and txt != ["<user>"]:
        id2text[tweet_id] = txt
    else: 
        pass

In [ ]:
print(f'the number of tweet acquired is:',{tweet_acquired})
print(f'the number of invalid tweet is:',{tweet_invalid})
print(f'the number of valid tweet is:',{tweet_valid})

the number of tweet acquired is: {50208}
the number of invalid tweet is: {7190}
the number of valid tweet is: {43018}


In [ ]:
all_keys = id2text.keys()
len(all_keys)

43010

In [ ]:
fo = open("/Users/siqiwu/Documents/Berkeley/Info-NLP/project/stance-dataset/dataset/wtwt_new.json", "r")
wtwt = json.load(fo)
fo.close()

In [ ]:
wtwt_obtained = []

In [ ]:
merger2target = json.load(open('/Users/siqiwu/Documents/Berkeley/Info-NLP/project/bias-stance/data/wtwt/merger2target.json'))

In [ ]:
for data in wtwt:
    if data["tweet_id"] in all_keys:
        d = data.copy()
        d["target"] = merger2target[d["merger"]].split()
        d["text"] = id2text[data["tweet_id"]]
        d["merger"] = d["merger"][4:] if "NEG_" in d["merger"] else d["merger"]
        assert d["merger"] in ['CVS_AET', 'CI_ESRX', 'AET_HUM', 'ANTM_CI', 'DIS_FOX']
        wtwt_obtained.append(d)

In [ ]:
fo = open("/Users/siqiwu/Documents/Berkeley/Info-NLP/project/stance-dataset/dataset/data_twtwt.json", "w+")
json.dump(wtwt_obtained, fo, indent=2)
fo.close()